In [ ]:
from dotenv import load_dotenv
import nest_asyncio
nest_asyncio.apply()
import os
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

In [23]:
from utils.helpers import get_doc_tools
from pathlib import Path


papers = [
    "batman.txt",
    "BTS.txt",
    "iPhone.txt"
]

paper_to_tools_dict = {}
for paper in papers:
    print(f"Getting tools for paper: {paper}")
    vector_tool, summary_tool = get_doc_tools(paper, Path(paper).stem)
    paper_to_tools_dict[paper] = [vector_tool, summary_tool]

Getting tools for paper: batman.txt
Getting tools for paper: BTS.txt
Getting tools for paper: iPhone.txt


In [24]:
initial_tools = [t for paper in papers for t in paper_to_tools_dict[paper]]

In [25]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo")

In [26]:
len(initial_tools)

6

In [27]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner

agent_worker = FunctionCallingAgentWorker.from_tools(
    initial_tools, 
    llm=llm, 
    verbose=True
)
agent = AgentRunner(agent_worker)

In [28]:
response = agent.query("Tell me about iphone")
print(str(response))

Added user message to memory: Tell me about iphone
=== Calling Function ===
Calling function: summary_tool_iPhone with args: {"input": "iPhone"}
=== Function Output ===
The iPhone is a line of smartphones developed and marketed by Apple that run iOS, the company's own mobile operating system. It was first announced by then–Apple CEO Steve Jobs on January 9, 2007, at Macworld 2007, and launched later that year. Since then, Apple has annually released new iPhone models and iOS versions. The iPhone has been credited with popularizing the slate smartphone form factor and creating a large market for smartphone apps through the App Store. As of January 1, 2024, more than 2.3 billion iPhones have been sold, making Apple the largest vendor of mobile phones in 2023.
=== LLM Response ===
The iPhone is a line of smartphones developed and marketed by Apple that run iOS, the company's own mobile operating system. It was first announced by then–Apple CEO Steve Jobs on January 9, 2007, at Macworld 20

In [29]:
response = agent.query("What's the diff between Iphone and batman?")
print(str(response))

Added user message to memory: What's the diff between Iphone and batman?
=== Calling Function ===
Calling function: summary_tool_iPhone with args: {"input": "Difference between iPhone and Batman"}
=== Function Output ===
The iPhone is a line of smartphones developed and marketed by Apple that run iOS, the company's own mobile operating system. It has been credited with popularizing the slate smartphone form factor and creating a large market for smartphone apps. On the other hand, Batman is a fictional superhero appearing in American comic books published by DC Comics. Batman is known for his detective skills, martial arts abilities, and use of advanced technology and gadgets to fight crime in Gotham City.
=== Calling Function ===
Calling function: summary_tool_batman with args: {"input": "Difference between iPhone and Batman"}
=== Function Output ===
iPhone is a popular smartphone developed by Apple Inc., known for its touchscreen interface and various features like apps, camera, and 

In [30]:
# define an "object" index and retriever over these tools
from llama_index.core import VectorStoreIndex
from llama_index.core.objects import ObjectIndex
all_tools = [t for paper in papers for t in paper_to_tools_dict[paper]]
obj_index = ObjectIndex.from_objects(
    all_tools,
    index_cls=VectorStoreIndex,
)

In [31]:
obj_retriever = obj_index.as_retriever(similarity_top_k=3)

In [32]:
tools = obj_retriever.retrieve(
    "Tell me about the advantage of Iphone over batman"
)

In [34]:
tools[1].metadata

ToolMetadata(description='Useful for summarization questions related to iPhone', name='summary_tool_iPhone', fn_schema=<class 'llama_index.core.tools.types.DefaultToolFnSchema'>, return_direct=False)

In [35]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner

agent_worker = FunctionCallingAgentWorker.from_tools(
    tool_retriever=obj_retriever,
    llm=llm, 
    system_prompt=""" \
You are an agent designed to answer queries over a set of given papers.
Please always use the tools provided to answer a question. Do not rely on prior knowledge.\

""",
    verbose=True
)
agent = AgentRunner(agent_worker)

In [37]:
response = agent.query(
    "Tell me about the important advantages of Iphone and "
    "compare it against spiderman"
)
print(str(response))

Added user message to memory: Tell me about the important advantages of Iphone and compare it against spiderman
=== Calling Function ===
Calling function: summary_tool_iPhone with args: {"input": "important advantages of iPhone"}
=== Function Output ===
The iPhone has been praised for its revolutionary design, user-friendly interface, and the introduction of innovative features such as multi-touch technology, Face ID, and Touch ID. Additionally, the iPhone ecosystem, including the App Store, has been highlighted for its high-quality apps and exclusive releases. The iPhone's premium positioning, security features, and strong privacy measures have also been significant advantages.
=== Calling Function ===
Calling function: summary_tool_batman with args: {"input": "advantages of Spiderman"}
=== Function Output ===
Spiderman is a superhero known for his agility, superhuman strength, ability to cling to walls, and a "spider-sense" that warns him of danger. He also possesses web-shooters tha